# Using Decomposition Objects

So far, the examples have covered how to use `BVDExperiment` to train models and calculate statistics (bias, variance, etc) in a single function.

However, the decomposition objects can be used in a standalone way, using predictions gathered from models trained and evaluated externally to this library.

## Cross Entropy & Squared Loss Decompositions

In the first example, we show how the `CrossEntropy` object can be used to get bias, variance and diversity for an ensemble using randomly generated data.

In [2]:
n_trials = 100
ensemble_size = 10
n_examples = 500
n_classes = 3

import numpy as np

# Labels are integers corresponding to the correct class
labels = np.random.choice(n_classes, size=(n_examples,))
# Predictions are in the form of one hot vectors
preds = np.random.uniform(0, 1 ,(n_trials, ensemble_size, n_examples, n_classes))
preds = preds / preds.sum(axis=3, keepdims=True)

from decompose import CrossEntropy

decomposition = CrossEntropy(preds, labels)

print("Bias-Variance Decomposition")
print("---------------------------")
print(f"expected risk: {decomposition.expected_ensemble_loss.mean()}")
print("\t=")
print(f"ensemble bias: {decomposition.ensemble_bias.mean()}")
print("\t+")
print(f"ensemble variance: {decomposition.ensemble_variance.mean()}")

print("\nBias-Variance-Diversity Decomposition")
print("-------------------------------------")
print(f"expected risk: {decomposition.expected_ensemble_loss.mean()}")
print("\t=")
print(f"average bias: {decomposition.average_bias.mean()}")
print("\t+")
print(f"average variance: {decomposition.average_variance.mean()}")
print("\t-")
print(f"diversity: {decomposition.diversity.mean()}")


Bias-Variance Decomposition
---------------------------
expected risk: 1.1313417337555043
	=
ensemble bias: 1.099645892902822
	+
ensemble variance: 0.03169584085268233

Bias-Variance-Diversity Decomposition
-------------------------------------
expected risk: 1.1313417337555043
	=
average bias: 1.1026982117921316
	+
average variance: 0.23516614892180485
	-
diversity: 0.20652262695843182


We can use the squared loss in the same way, with the only difference being the shape of the predictions array

In [2]:
n_trials = 100
ensemble_size = 10
n_examples = 500
n_classes = 3

import numpy as np

# real-valued labels
labels = np.random.normal(0.5, 1., size=(n_examples,))
# real-values predictions
preds = np.random.normal(0, 1 ,(n_trials, ensemble_size, n_examples))

from decompose import SquaredLoss

decomposition = SquaredLoss(preds, labels)
print("Bias-Variance Decomposition")
print("---------------------------")
print(f"expected risk: {decomposition.expected_ensemble_loss.mean()}")
print("\t=")
print(f"ensemble bias: {decomposition.ensemble_bias.mean()}")
print("\t+")
print(f"ensemble variance: {decomposition.ensemble_variance.mean()}")

print("\nBias-Variance-Diversity Decomposition")
print("-------------------------------------")
print(f"expected risk: {decomposition.expected_ensemble_loss.mean()}")
print("\t=")
print(f"average bias: {decomposition.average_bias.mean()}")
print("\t+")
print(f"average variance: {decomposition.average_variance.mean()}")
print("\t-")
print(f"diversity: {decomposition.diversity.mean()}")

Bias-Variance Decomposition
---------------------------
expected risk: 1.4113203905210603
	=
ensemble bias: 1.3112493974383124
	+
ensemble variance: 0.10007099308274785

Bias-Variance-Diversity Decomposition
-------------------------------------
expected risk: 1.4113203905210603
	=
average bias: 1.3206010421925793
	+
average variance: 0.9922307885441782
	-
diversity: 0.9015114402156975


## 0-1 Loss Effect Decomposition

The effect decomposition for the 0-1 loss works in the same way but takes class label predictions:

In [3]:
n_trials = 100
ensemble_size = 10
n_examples = 500
n_classes = 3

import numpy as np

# Labels are integers corresponding to the correct class
labels = np.random.choice(n_classes, size=(n_examples,))
# Predictions are in the form of class labels.
# We randomly generate predictions so that half of all member predictions are correct (in an i.i.d. way)
preds = np.broadcast_to(labels, shape=(n_trials, ensemble_size, n_examples))
incorrect_preds = np.random.choice([0, 1], size=(n_trials, ensemble_size, n_examples), p=[0.5, 0.5])
preds = (preds + incorrect_preds * np.random.choice([1, 2], size=(n_trials, ensemble_size, n_examples))) % n_classes

from decompose import ZeroOneLoss

decomposition = ZeroOneLoss(preds, labels)


print(f"bias-variance decomposition")
print(f"---------------------------")
print(f"expected risk: {decomposition.expected_ensemble_loss.mean()}")
print("\t=")
print(f"ensemble bias: {decomposition.ensemble_bias.mean()}")
print("\t+")
print(f"ensemble variance: {decomposition.ensemble_variance_effect.mean()}")

print(f"\nbias-variance-diversity effect decomposition")
print(f"--------------------------------------------")
print(f"expected risk: {decomposition.expected_ensemble_loss.mean()}")
print("\t=")
print(f"average bias: {decomposition.average_bias.mean()}")
print("\t+")
print(f"average variance: {decomposition.average_variance_effect.mean()}")
print("\t-")
print(f"diversity: {decomposition.diversity_effect.mean()}")




bias-variance decomposition
---------------------------
expected risk: 0.27218000000000003
	=
ensemble bias: 0.0
	+
ensemble variance: 0.27218000000000003

bias-variance-diversity effect decomposition
--------------------------------------------
expected risk: 0.27218000000000003
	=
average bias: 0.0032
	+
average variance: 0.496404
	-
diversity: 0.227424


## Example Experiment

In this example, we show how a simple experiment script may look. We train a collection of ensembles, each on 90% of the total available training data. Collecting the predictions from each ensemble member in each trial, we are then able to create an instance of the `CrossEntropy` decomposition class from which we get bias, variance and diversity estimates.



In [6]:
from decompose.data_utils import load_standard_dataset
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from decompose import CrossEntropy

train_data, train_labels, test_data, test_labels = load_standard_dataset("digits", frac_training=0.5, normalize_data=True)



n_trials = 10
ensemble_size = 5

# We construct a numpy array with the predictions of ensemble members over numerous trials,
# the shape of the numpy array is (n_trials, ensemble_size, test_data_size, n_classes)
# for regression, the n_classes dimension is omitted.
model_outputs = np.zeros((n_trials, ensemble_size, test_data.shape[0], 10))


for t_idx in range(n_trials):
    subsample_size = int(train_data.shape[0] * 0.9)
    subsample_indices = np.random.permutation(train_data.shape[0])[:subsample_size]
    train_data_sample = train_data[subsample_indices, :]
    train_labels_sample = train_labels[subsample_indices]
    # ignore covergence warnings from MLPClassifier
    import warnings
    warnings.filterwarnings("ignore")
    ensemble = BaggingClassifier(n_estimators=ensemble_size,
                                 base_estimator=MLPClassifier(hidden_layer_sizes=20,
                                                              max_iter=100,
                                                              learning_rate_init=0.001,
                                                              alpha=0.1,
                                                              momentum=0,
                                                              solver="adam"))
    ensemble.fit(train_data_sample, train_labels_sample)
    for e_idx, estimator in enumerate(ensemble.estimators_):
        model_outputs[t_idx, e_idx, :, :] = estimator.predict_proba(test_data)

bvd_decomposition = CrossEntropy(model_outputs, test_labels)


# Add noise to deal with fact geometric mean isn't defined for zero
model_outputs = (1 - 1e-9) * model_outputs + 1e-10 * np.ones_like(model_outputs)

# We can now print out quantities from the decomposition for instance the diversity is
print(bvd_decomposition.diversity.mean())

0.06893953762076134
